In [1]:
import numpy as np
import os
import sys
import collections
import struct

# Use your path here
# path = '/Users/meghna/Desktop/PhD-NeRF/colmap_fernfordatacollection'
# path = '/Users/meghna/Desktop/PhD-NeRF/my_plant'
# path = '/Users/meghna/Desktop/PhD-NeRF/headphones'
# path = '/Users/meghna/Desktop/PhD-NeRF/teabox'
# path = '/Users/meghna/Desktop/PhD-NeRF/my_plant_ldsp'
# path = '/Users/meghna/Desktop/PhD-NeRF/buddha_known_cp'
# path = '/Users/meghna/Desktop/PhD-NeRF/reading_avg'
path = '/Users/meghna/Desktop/PhD-NeRF/reading_avg'

## Camera Data - Intrinsics

In [2]:
Camera = collections.namedtuple(
    "Camera", ["id", "model", "width", "height", "params"])

def read_cameras_text(path):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::WriteCamerasText(const std::string& path)
        void Reconstruction::ReadCamerasText(const std::string& path)
    """
    cameras = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                camera_id = int(elems[0])
                model = elems[1]
                width = int(elems[2])
                height = int(elems[3])
                params = np.array(tuple(map(float, elems[4:])))
                cameras[camera_id] = Camera(id=camera_id, model=model,
                                            width=width, height=height,
                                            params=params)
    return cameras

In [3]:
camdata = read_cameras_text(path+'/cameras.txt')
list_of_keys = list(camdata.keys())
cam = camdata[list_of_keys[0]]
print( 'Cameras', len(cam))

h, w, f = cam.height, cam.width, cam.params[0]
# w, h, f = factor * w, factor * h, factor * f
hwf = np.array([h,w,f]).reshape([3,1])
print('HWF: ', hwf)

Cameras 5
HWF:  [[ 512.  ]
 [ 612.  ]
 [3765.54]]


## Image Data - Rot and trans matx

### Helper func

In [4]:
def qvec2rotmat(qvec):
    return np.array([
        [1 - 2 * qvec[2]**2 - 2 * qvec[3]**2,
         2 * qvec[1] * qvec[2] - 2 * qvec[0] * qvec[3],
         2 * qvec[3] * qvec[1] + 2 * qvec[0] * qvec[2]],
        [2 * qvec[1] * qvec[2] + 2 * qvec[0] * qvec[3],
         1 - 2 * qvec[1]**2 - 2 * qvec[3]**2,
         2 * qvec[2] * qvec[3] - 2 * qvec[0] * qvec[1]],
        [2 * qvec[3] * qvec[1] - 2 * qvec[0] * qvec[2],
         2 * qvec[2] * qvec[3] + 2 * qvec[0] * qvec[1],
         1 - 2 * qvec[1]**2 - 2 * qvec[2]**2]])


def rotmat2qvec(R):
    Rxx, Ryx, Rzx, Rxy, Ryy, Rzy, Rxz, Ryz, Rzz = R.flat
    K = np.array([
        [Rxx - Ryy - Rzz, 0, 0, 0],
        [Ryx + Rxy, Ryy - Rxx - Rzz, 0, 0],
        [Rzx + Rxz, Rzy + Ryz, Rzz - Rxx - Ryy, 0],
        [Ryz - Rzy, Rzx - Rxz, Rxy - Ryx, Rxx + Ryy + Rzz]]) / 3.0
    eigvals, eigvecs = np.linalg.eigh(K)
    qvec = eigvecs[[3, 0, 1, 2], np.argmax(eigvals)]
    if qvec[0] < 0:
        qvec *= -1
    return qvec

In [5]:
BaseImage = collections.namedtuple(
    "Image", ["id", "qvec", "tvec", "camera_id", "name", "xys", "point3D_ids"])

class Image(BaseImage):
    def qvec2rotmat(self):
        return qvec2rotmat(self.qvec)

def read_images_text(path):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::ReadImagesText(const std::string& path)
        void Reconstruction::WriteImagesText(const std::string& path)
    """
    images = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                image_id = int(elems[0])
                qvec = np.array(tuple(map(float, elems[1:5])))
                tvec = np.array(tuple(map(float, elems[5:8])))
                camera_id = int(elems[8])
                image_name = elems[9]
                elems = fid.readline().split()
                xys = np.column_stack([tuple(map(float, elems[0::3])),
                                       tuple(map(float, elems[1::3]))])
                point3D_ids = np.array(tuple(map(int, elems[2::3])))
                images[image_id] = Image(
                    id=image_id, qvec=qvec, tvec=tvec,
                    camera_id=camera_id, name=image_name,
                    xys=xys, point3D_ids=point3D_ids)
    return images

In [6]:
imdata = read_images_text(path+'/images.txt')
print(imdata)

{1: Image(id=1, qvec=array([ 0.361318,  0.598015,  0.607324, -0.378134]), tvec=array([ -81.4297,  -14.6731, 1623.33  ]), camera_id=1, name='001.jpg', xys=array([[290.128 , 154.463 ],
       [310.953 , 152.486 ],
       [317.26  , 157.792 ],
       [325.031 , 146.576 ],
       [325.031 , 146.576 ],
       [329.348 , 149.225 ],
       [307.053 , 161.72  ],
       [306.607 , 174.33  ],
       [325.753 , 166.609 ],
       [328.569 , 167.855 ],
       [331.537 , 168.133 ],
       [337.663 , 165.952 ],
       [338.371 , 168.52  ],
       [218.352 , 187.209 ],
       [228.919 , 189.62  ],
       [289.403 , 184.729 ],
       [290.028 , 188.818 ],
       [308.896 , 182.733 ],
       [338.696 , 180.328 ],
       [207.232 , 207.019 ],
       [238.372 , 195.817 ],
       [236.321 , 197.865 ],
       [237.058 , 200.941 ],
       [242.089 , 200.083 ],
       [291.788 , 196.022 ],
       [298.702 , 203.619 ],
       [309.12  , 196.627 ],
       [309.12  , 196.627 ],
       [312.117 , 197.274 ],
     

In [7]:
w2c_mats = []
bottom = np.array([0,0,0,1.]).reshape([1,4])
    
names = [imdata[k].name for k in imdata]
print( 'Images #', len(names))
perm = np.argsort(names)
for k in imdata:
    im = imdata[k]
    R = im.qvec2rotmat()
    t = im.tvec.reshape([3,1])
    m = np.concatenate([np.concatenate([R, t], 1), bottom], 0)
    w2c_mats.append(m)
    
w2c_mats = np.stack(w2c_mats, 0)
c2w_mats = np.linalg.inv(w2c_mats)
    
poses = c2w_mats[:, :3, :4].transpose([1,2,0])
poses = np.concatenate([poses, np.tile(hwf[..., np.newaxis], [1,1,poses.shape[-1]])], 1)

# must switch to [-u, r, -t] from [r, -u, t], NOT [r, u, -t]
poses = np.concatenate([poses[:, 1:2, :], poses[:, 0:1, :], -poses[:, 2:3, :], poses[:, 3:4, :], poses[:, 4:5, :]], 1)

Images # 20


## Points3d data

In [8]:
Point3D = collections.namedtuple(
    "Point3D", ["id", "xyz", "rgb", "error", "image_ids", "point2D_idxs"])

def read_points3D_text(path):
    """
    see: src/base/reconstruction.cc
        void Reconstruction::ReadPoints3DText(const std::string& path)
        void Reconstruction::WritePoints3DText(const std::string& path)
    """
    points3D = {}
    with open(path, "r") as fid:
        while True:
            line = fid.readline()
            if not line:
                break
            line = line.strip()
            if len(line) > 0 and line[0] != "#":
                elems = line.split()
                point3D_id = int(elems[0])
                xyz = np.array(tuple(map(float, elems[1:4])))
                rgb = np.array(tuple(map(int, elems[4:7])))
                error = float(elems[7])
                image_ids = np.array(tuple(map(int, elems[8::2])))
                point2D_idxs = np.array(tuple(map(int, elems[9::2])))
                points3D[point3D_id] = Point3D(id=point3D_id, xyz=xyz, rgb=rgb,
                                               error=error, image_ids=image_ids,
                                               point2D_idxs=point2D_idxs)
    return points3D

In [9]:
pts3d = read_points3D_text(path+'/points3D.txt')
print(pts3d)

{1: Point3D(id=1, xyz=array([92.4967, 84.2466, 67.0575]), rgb=array([91, 89, 89]), error=0.935443, image_ids=array([ 2, 20,  1]), point2D_idxs=array([6, 6, 7])), 2: Point3D(id=2, xyz=array([91.0762, 96.9181, 63.6011]), rgb=array([38, 34, 35]), error=1.16688, image_ids=array([ 2, 20,  1]), point2D_idxs=array([16, 10, 18])), 3: Point3D(id=3, xyz=array([66.5238, 55.2866, 44.0735]), rgb=array([116, 119, 123]), error=0.74353, image_ids=array([2, 1, 3]), point2D_idxs=array([131,  21, 139])), 4: Point3D(id=4, xyz=array([109.24   ,  93.3455 ,   6.36154]), rgb=array([127, 139, 163]), error=0.67839, image_ids=array([20,  1, 19]), point2D_idxs=array([115, 121, 129])), 5: Point3D(id=5, xyz=array([107.47   ,  93.5142 ,   3.50884]), rgb=array([124, 134, 163]), error=0.0215138, image_ids=array([20,  1]), point2D_idxs=array([116, 122])), 6: Point3D(id=6, xyz=array([63.6992, 48.1791, 48.8969]), rgb=array([108, 112, 118]), error=0.881818, image_ids=array([2, 1, 3]), point2D_idxs=array([196, 142, 188])),

In [10]:
pts_arr = []
vis_arr = []
for k in pts3d:
    pts_arr.append(pts3d[k].xyz)
    cams = [0] * poses.shape[-1]
    for ind in pts3d[k].image_ids:
#         if len(cams) < ind - 1:
#             print('ERROR: the correct camera poses for current points cannot be accessed')
#             return
        cams[ind-1] = 1
    vis_arr.append(cams)
    
pts_arr = np.array(pts_arr)
vis_arr = np.array(vis_arr)
print( 'Points', pts_arr.shape, 'Visibility', vis_arr.shape )
    
zvals = np.sum(-(pts_arr[:, np.newaxis, :].transpose([2,0,1]) - poses[:3, 3:4, :]) * poses[:3, 2:3, :], 0)
valid_z = zvals[vis_arr==1]
print( 'Depth stats', valid_z.min(), valid_z.max(), valid_z.mean() )

Points (177, 3) Visibility (177, 20)
Depth stats 1498.3600979259813 1568.5784565179856 1528.6610185211227


In [11]:
save_arr = []
for i in perm:
    vis = vis_arr[:, i]
    zs = zvals[:, i]
    zs = zs[vis==1]
    close_depth, inf_depth = np.percentile(zs, .1), np.percentile(zs, 99.9)
    # print( i, close_depth, inf_depth )
        
    save_arr.append(np.concatenate([poses[..., i].ravel(), np.array([close_depth, inf_depth])], 0))
save_arr = np.array(save_arr)

In [12]:
np.save('poses_bounds.npy', save_arr)

In [13]:
np.load('poses_bounds.npy')

array([[ 4.53124433e-01, -2.36549203e-02,  8.91132916e-01,
         1.45132532e+03,  5.12000000e+02, -1.21393211e-03,
         9.99630155e-01,  2.71528692e-02,  1.25459839e+02,
         6.12000000e+02, -8.91446001e-01, -1.33857654e-02,
         4.52927903e-01,  7.21081178e+02,  3.76554000e+03,
         1.50651592e+03,  1.55256066e+03],
       [ 4.36805251e-01, -2.87714483e-01,  8.52305069e-01,
         1.39126651e+03,  5.12000000e+02,  1.19362309e-01,
         9.57632588e-01,  2.62095581e-01,  4.87692504e+02,
         6.12000000e+02, -8.91602737e-01, -1.27508256e-02,
         4.52641870e-01,  7.20456519e+02,  3.76554000e+03,
         1.50158565e+03,  1.55104741e+03],
       [ 3.76640771e-01, -5.75043172e-01,  7.26269810e-01,
         1.19739292e+03,  5.12000000e+02,  2.51765131e-01,
         8.18036609e-01,  5.17137104e-01,  8.81153218e+02,
         6.12000000e+02, -8.91491115e-01, -1.19253265e-02,
         4.52881967e-01,  7.20990922e+02,  3.76554000e+03,
         1.50552379e+03,  1.5